In [2]:
m.table<- read.table(file = "dseqmerged.txt",header=T)
m2.table<-m.table[,c(1:4,9:12)]
mt<-na.omit(m2.table) 
dim(m.table)
dim(m2.table)
dim(mt)
head(mt)

[1] 39663    12

[1] 39663     8

[1] 16581     8

,pval641,FC641,baseMean641,lfcSE641,pval110,FC110,baseMean110,lfcSE110
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,0.0213434747,-0.8539164,5.440560,0.3219830,0.178717429,-0.1839214,78.850310,0.1136161
A1CF,0.0049613962,0.9794048,10.683128,0.3063447,0.274960101,0.4352421,6.380028,0.3245492
A2M,0.6493664834,-0.1293646,1268.096057,0.2146932,0.146841451,0.4157708,41.067909,0.2396970
A2ML1,0.0103445995,0.7072489,16.479937,0.2411158,0.004946548,0.7672777,36.093265,0.2422568
A3GALT2,0.0003460403,1.0940383,17.347226,0.2720013,0.002882060,2.0026141,11.245042,0.5981626
A4GALT,0.0589380905,-1.0663099,5.851756,0.4851022,0.721833019,0.3067707,3.921329,0.6392574


In [ ]:
# Marker investigation
library(dplyr)
sdirection_mt <- mt %>%
  filter(
    (FC641 > 0 & FC110 > 0) | 
    (FC641 < 0 & FC110 < 0)
  )

sig_mt <- sdirection_mt %>%
  filter(
    (pval641 < 0.05 & pval110 < 0.05 )
  )
Filtered_mt <- sig_mt %>%
  filter(
    (FC641 >= 1 & FC110 >= 1 ) | 
    (FC641 <= -1 & FC110 <= -1)
  )
x <- Filtered_mt %>%
  filter(
    (pval641 < 0.01 & pval110 < 0.01),
    (FC641 >= 2 & FC110 >= 2 ) | 
    (FC641 <= -2 & FC110 <= -2 )  )
x2 <- Filtered_mt %>%
  filter(
    (pval641 < 0.01 & pval110 < 0.01),
    (FC641 >= 2 & FC110 >= 2 ) | 
    (FC641 <= -2 & FC110 <= -2 ),
    (baseMean641> 20 & baseMean110>20),
    (lfcSE641<0.5 & lfcSE110<0.5)  )
#Negative Log2 Fold Change: downregulation in the "Severe" condition.
downincovidx2<- x2 %>%
  filter(FC641 <= -2 & FC110 <= -2 )
#Positive Log2 Fold Change: upregulation in the "Severe" condition.
upincovidx2<- x2 %>%
  filter(FC641 >= 2 & FC110 >= 2 )

In [13]:
InfvsCon<- read.table(file = "GSE163151/DEseq All influenza_vs_control result.txt",header=T,row.names=1,sep=",")
BacvsCon<-read.table(file = "GSE163151/DEseq All bacterial_vs_control result.txt",header=T,row.names=1,sep=",")

genes_in_infDEG <- rownames(x2)[rownames(x2) %in% rownames(InfvsCon)]
length(genes_in_infDEG)
genes_in_bacDEG <- rownames(x2)[rownames(x2) %in% rownames(BacvsCon)]
length(genes_in_bacDEG)

inf <- InfvsCon %>%
  filter  ((padj < 0.05 ) ,
           (abs(log2FoldChange)>1)
           )
dim(inf)
bac <- BacvsCon %>%
  filter  ((padj < 0.05 ) ,
           (abs(log2FoldChange)>1)
           )
genes_in_infDEG <- rownames(x2)[rownames(x2) %in% rownames(inf)]
genes_in_bacDEG <- rownames(x2)[rownames(x2) %in% rownames(bac)]
# common genes among x2, inf, and bac
common_x2_inf <- intersect(rownames(x2), rownames(inf))
common_all <- intersect(common_x2_inf, rownames(bac))
print("common gene in all")
common_all
# Genes that are in x2 and not in inf or bac
combined_inf_bac <- union(rownames(inf), rownames(bac))
unique_x2 <- setdiff(rownames(x2), combined_inf_bac)
print("unique gene in COVID-19")
unique_x2

[1] "common gene in all"


[1] "GPR84"    "LRRN3"    "MCEMP1"   "MMP9"     "MS4A4A"   "SERPING1"

[1] "unique gene in COVID-19"


[1] "GALNT14" "IGFBP2"  "INSC"    "ITGA7"   "MAOB"    "MZB1"    "NRN1"   
 [8] "PPARG"   "PTGFR"   "SDC1"    "SPATC1"  "ZNF608"

In [14]:
# Proteomic validation
protein_genes<- read.table(file = "protein_genes.txt",header=F)
dim(protein_genes)
validated_gene<-intersect(rownames(x2), protein_genes[,1])
validated_gene

[1] 3043    1

[1] "ANKRD22"  "C1QC"     "HP"       "IGFBP2"   "MAOB"     "MCEMP1"   "MMP9"    
[8] "MZB1"     "SERPING1"

[1] "IGFBP2" "MAOB"   "MZB1"

In [23]:
markers <- c('GALNT14', 'IGFBP2', 'INSC', 'ITGA7', 'MAOB', 'MZB1', 'NRN1', 'PPARG', 'PTGFR', 'SDC1', 'SPATC1', 'ZNF608')
x2[markers,]


,pval641,FC641,baseMean641,lfcSE641,pval110,FC110,baseMean110,lfcSE110
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GALNT14,1.927596e-10,2.022375,273.68691,0.2911207,1.824760e-21,3.504879,532.88721,0.3462060
IGFBP2,1.267201e-07,2.308800,34.17039,0.3967617,1.914795e-06,2.592249,121.25824,0.4965969
INSC,4.799610e-19,2.278782,32.37208,0.2364258,3.768578e-12,2.464576,65.72010,0.3296101
ITGA7,9.470411e-14,2.549581,58.86397,0.3154832,2.752413e-24,4.908586,663.55402,0.4518599
MAOB,7.672572e-07,2.164309,23.29413,0.3960549,7.041021e-15,3.940736,95.14941,0.4727556
MZB1,7.914912e-15,2.104174,274.89580,0.2499820,6.015483e-21,3.466877,4879.44531,0.3470633
NRN1,1.669486e-07,2.035167,39.39361,0.3529286,4.424443e-09,2.484141,118.07625,0.3896391
PPARG,9.010345e-13,2.156509,46.51096,0.2782294,8.714037e-16,3.312816,180.77109,0.3852625
PTGFR,1.234349e-14,2.391181,39.57852,0.2860539,1.053493e-08,2.925424,87.66007,0.4699926


In [ ]:
**Annotation**

In [18]:
library(org.Hs.eg.db)
library(AnnotationDbi)
gene_symbols <- markers 
# Expanded annotations :"ENTREZID", "SYMBOL", "GENENAME", "CHR", "MAP", "GO", "ENSEMBL", "UNIPROT"
result <- select(org.Hs.eg.db, 
                          keys = gene_symbols, 
                          columns = c( "SYMBOL", "GENENAME","UNIPROT", "CHR"),
                          keytype = "SYMBOL")
print(result)


Warning message in .deprecatedColsMessage():
"Accessing gene location information via 'CHR','CHRLOC','CHRLOCEND' is
  deprecated. Please use a range based accessor like genes(), or select()
  with columns values like TXCHROM and TXSTART on a TxDb or OrganismDb
  object instead."
'select()' returned 1:many mapping between keys and columns



    SYMBOL                                         GENENAME    UNIPROT CHR
1  GALNT14 polypeptide N-acetylgalactosaminyltransferase 14     Q96FL9   2
2  GALNT14 polypeptide N-acetylgalactosaminyltransferase 14     B7Z5C5   2
3   IGFBP2     insulin like growth factor binding protein 2     P18065   2
4   IGFBP2     insulin like growth factor binding protein 2     C9JMY1   2
5     INSC         INSC spindle orientation adaptor protein     Q1MX18  11
6    ITGA7                         integrin subunit alpha 7     Q13683  12
7    ITGA7                         integrin subunit alpha 7     Q4LE35  12
8     MAOB                              monoamine oxidase B     P27338   X
9     MAOB                              monoamine oxidase B     B7Z242   X
10    MZB1     marginal zone B and B1 cell specific protein     Q8WU39   5
11    NRN1                                       neuritin 1     Q9NPD7   6
12    NRN1                                       neuritin 1 A0A087WWT2   6
13   PPARG peroxisome pro

In [ ]:
GO_result <- select(org.Hs.eg.db, 
                          keys = gene_symbols, 
                          columns = c( "SYMBOL", "GO"),
                          keytype = "SYMBOL")
print(GO_result)